In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [5]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  x = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [8]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [9]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [10]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [11]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
    
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [14]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [18]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [19]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_is wheeled', 'feat_frame material:', 'feat_walmart no.',
       'feat_productgroup', 'feat_band length (in.)', 'feat_clothing type',
       'feat_designed for', 'feat_shade', 'feat_full product manual',
       'feat_ul safety listing'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [25]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [37]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [0]:
feats = ['']

In [39]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.29367863337959, 4.248588007694293)

In [60]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_fabrication',
 'feat_location - city/state',
 'feat_shoe category',
 'feat_recommended location',
 'feat_multi pack indicator',
 'feat_location - country',
 'feat_clothing category',
 'feat_catalog',
 'feat_certifications and listings',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_material_cat',
 'feat_manufacturer part number_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_model no._cat',
 'feat_cm_cat',
 'feat_fabrication_cat',
 'feat_number of compartments_cat',
 'feat_shirt size_cat',
 'feat_international shipping?_cat',
 'feat_weather resistant_cat',
 'feat_country of origin assembly:_cat',
 'feat_vendor description_cat',
 'feat_navy blue_cat',
 'feat_part type_cat',
 'feat_net weight_cat',
 'feat_special features_cat',
 'feat_location - city/state_cat',
 'feat_cushioning_cat',
 'feat_ean_cat',
 'feat_euro size_cat',
 'feat_front_cat',
 'feat_airport friendly_cat',
 'feat_protects against_cat',
 'feat_finish_cat',
 'feat_a

In [0]:
feats = ['brand_cat', 'feat_metal type_cat', 'feat_shape_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']


model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [66]:
X = df[ feats ].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.25352779119296, 4.285903138481129)


Weight,Feature
0.2557 ± 0.0080,brand_cat
0.1032 ± 0.0075,feat_material_cat
0.0252 ± 0.0026,feat_gender_cat
0.0175 ± 0.0015,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0091 ± 0.0015,feat_metal type_cat
0.0033 ± 0.0012,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [56]:
df[ df['brand'] == 'nike' ].features_parsed.sample(5).values

array([{'style': 'basketball shoes', 'condition': 'new with box'},
       {'size': '10', 'model number': '705149 010', 'shoe width': 'medium', 'material': 'mesh', 'sport': 'running shoes', 'occasion': 'running shoes', 'gender': 'men', 'shoe size': '8.5 us men', 'shoe category': "men's shoes", 'fabric content': 'mesh', 'color': 'black/anthracite/black', 'model': '705149 010', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '705149 010', 'brand': 'nike', 'age group': 'men'},
       {'sport': 'soccer cleats', 'occasion': 'soccer cleats', 'material': 'synthetic', 'gender': 'men', 'shoe size': '12 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'synthetic', 'color': 'wolf grey/hyper pink/black/blk', 'model': '651635 060', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'soccer shoes', 'manufacturer part number': '651635 060', 'variant group id': '

In [67]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [68]:
ls

data/  day5.ipynb  HelloGithub.ipynb  LICENSE  matrix_one/  README.md
